In [98]:
import warnings
warnings.filterwarnings('ignore')

In [99]:
from numpy.random import seed
seed(42)

In [100]:
import os
import pandas as pd
import math
import numpy as np

In [101]:
! pip install wget


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [102]:
! pip install holidays


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [103]:
import wget
import os.path

if(not os.path.exists('finale.csv')):
    wget.download('https://data.4tu.nl/ndownloader/files/23993303')

In [104]:
df_finale = pd.read_csv('finale.csv', index_col=0).reset_index()

In [105]:
df_finale.columns

Index(['Case ID', 'Activity', 'Resource', 'Complete Timestamp', 'Variant',
       'Variant index', 'Variant.1', 'seriousness', 'customer', 'product',
       'responsible_section', 'seriousness_2', 'service_level', 'service_type',
       'support_section', 'workgroup'],
      dtype='object')

In [106]:
df_finale.shape

(21348, 16)

String to datetime conversion

In [107]:
df_finale['Complete Timestamp'] = pd.to_datetime(df_finale['Complete Timestamp'])

In [108]:
from datetime import date
import holidays

it_holidays = holidays.Italy()

In [109]:
df_finale['Date'] = df_finale['Complete Timestamp'].dt.strftime("%Y-%m-%d")
dates = df_finale['Date'].unique()

In [110]:
holidays_helpdesk = []

for date in dates:
    if(date in it_holidays):
        holidays_helpdesk.append(date)    

len(holidays_helpdesk)

30

Sorting case ids by early timestamp

In [111]:
sorted_time_cases = df_finale.sort_values('Complete Timestamp')['Case ID'].drop_duplicates().values

#### Feature creation
- **Duration**: Duration of previous activity
- **Passed Time**: Passed time between first activity and current one
- **Time to conclusion**: Remaining time to process finishes

In [126]:
from tqdm import tqdm

data = []

for case in tqdm(sorted_time_cases):
    #print(case)
    df_case = df_finale[df_finale['Case ID'] == case]
    df_case['Complete Timestamp Shift'] = df_case['Complete Timestamp'].shift(1)
    
    ##datetime
    df_case['Duration'] = df_case['Complete Timestamp'] - df_case['Complete Timestamp Shift']
    
    ## colapse
    df_case = (df_case.groupby((df_case['Activity'] != df_case['Activity'].shift(1)).cumsum(), as_index=False)
               .agg(CaseID=("Case ID", "first"), Activity=("Activity", "first"), Duration=("Duration", "sum")
                #Reward_avg=("Duration Float", "mean")
                )
              )
    
    ## day as float 
    df_case['Duration Float'] = df_case['Duration'] / pd.to_timedelta(1, unit='D')
    df_case['Duration Float'] = df_case['Duration Float'].fillna(0).round(4)
    
    time_to_conclusion = []
    total_time = df_case['Duration Float'].sum()
    
    for time in df_case['Duration Float']:
        total_time = total_time - time
        time_to_conclusion.append(total_time)
        
    df_case['Time to conclusion'] = time_to_conclusion
    df_case['Time to conclusion'] = df_case['Time to conclusion'].round(4)
    
    df_case['Passed Time'] = df_case['Time to conclusion'].iloc[::-1].values
    df_case['Passed Time'] = df_case['Passed Time'].round(4)
    
    df_case['Step'] = np.arange(1, df_case.shape[0]+1)    
    
    data.append(df_case)

100%|██████████| 4580/4580 [02:43<00:00, 28.05it/s]


In [127]:
df_case

,CaseID,Activity,Duration,Duration Float,Time to conclusion,Passed Time,Step
0,Case 1735,Assign seriousness,0 days 00:00:00,0.0000,33.6809,-0.0000,1
1,Case 1735,Require upgrade,17 days 23:00:54,17.9590,15.7219,15.0002,2
2,Case 1735,Resolve ticket,0 days 17:19:14,0.7217,15.0002,15.7219,3
3,Case 1735,Closed,15 days 00:00:18,15.0002,-0.0000,33.6809,4


In [128]:
"""
(df_case.groupby((df_case['Activity'] != df_case['Activity'].shift(1)).cumsum(), as_index=False)
   .agg(Activity=("Activity", "first"), 
        Duration=("Duration", "sum") 
        #Reward_avg=("Duration Float", "mean")
       )
)
"""

'\n(df_case.groupby((df_case[\'Activity\'] != df_case[\'Activity\'].shift(1)).cumsum(), as_index=False)\n   .agg(Activity=("Activity", "first"), \n        Duration=("Duration", "sum") \n        #Reward_avg=("Duration Float", "mean")\n       )\n)\n'

In [129]:
df_case['Complete Timestamp']

KeyError: 'Complete Timestamp'

In [ ]:
df_case['Complete Timestamp'].shift(1)

Example:

In [130]:
df_case[['Step', 'Duration Float', 'Passed Time', 'Time to conclusion']]

,Step,Duration Float,Passed Time,Time to conclusion
0,1,0.0000,-0.0000,33.6809
1,2,17.9590,15.0002,15.7219
2,3,0.7217,15.7219,15.0002
3,4,15.0002,33.6809,-0.0000


In [131]:
appended_data = pd.concat(data)

In [132]:
pd.DataFrame(appended_data).to_csv('finale_time_features_colapsed.csv', index=False)

In [ ]:
df_final = pd.read_csv('finale_time_features_colapsed.csv')

In [ ]:
df_final.shape

In [ ]:
last_activities = []
closed_cases = []
not_closed_cases = []

for name, group in df_final.groupby('Case ID'):
    if(group.tail(1)['Activity'].values[0] == 'Closed'):
        closed_cases.append(name)
    else:
        not_closed_cases.append(name)
    last_activities.append(group.tail(1)['Activity'].values[0])    

In [ ]:
from collections import Counter

Counter(last_activities)

In [ ]:
data_case = [
    [len(closed_cases), df_final[df_final['Case ID'].isin(closed_cases)].shape[0]],
    [len(not_closed_cases), df_final[df_final['Case ID'].isin(not_closed_cases)].shape[0]],
    []
]

In [ ]:
pd.DataFrame(data_case, columns=['# of Cases', '# Activities'])

In [ ]:
df_final[df_final['Case ID'] == 'Case 10'][['Duration Float','Time to conclusion','Passed Time','Step']]

In [ ]:
df_final[df_final['Case ID'] == 'Case 67'][['Duration Float','Time to conclusion','Passed Time','Step']]